# Latent Analysis for Protein-Ligand Docking Poses

This notebook provides comprehensive analysis of latent vectors from protein-ligand docking optimization data.

## Analysis Goals
1. **Statistical Analysis**: Understand the distribution and properties of latent vectors
2. **Quality Assessment**: Analyze the relationship between latent space and pose quality
3. **Correlation Analysis**: Find which latent dimensions are most predictive of quality
4. **Visualization**: Create intuitive plots to understand the data structure
5. **Generation Analysis**: Track how optimization progresses across generations


## 1. Setup and Imports


In [1]:
import os
import pickle
import glob
import time
from datetime import datetime as dt
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from typing import List, Union, Tuple, Optional, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Schrödinger imports (if available)
try:
    from schrodinger.structure import StructureReader
    print("✅ Schrödinger Python API loaded successfully!")
    SCHRODINGER_AVAILABLE = True
except ImportError:
    print("⚠️  Schrödinger Python API not available in this environment")
    print("   To use Schrödinger features, run this notebook with:")
    print("   $SCHRODINGER/suites2025-3/run jupyter notebook")
    SCHRODINGER_AVAILABLE = False

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Imports successful!")
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Pandas version: {pd.__version__}")


Unable to revert mtime: /Library/Fonts


✅ Schrödinger Python API loaded successfully!
Imports successful!
NumPy version: 1.24.2
PyTorch version: 2.0.1
Pandas version: 1.5.3


## 2. Data Loading Functions


In [23]:
def load_latents(files: str) -> Tuple[torch.Tensor, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Loads latent vectors from .pkl files (based on Yang's example.ipynb)

    Returns:
        z_list: torch.Tensor; list of latent vectors
        E_list: np.ndarray; list of energies for each latent vector (same order)
        sr_list: np.ndarray; list of protein site rmsd for each latent vector (same order)
        lr_list: np.ndarray; list of ligand rmsd for each latent vector (same order)
        gen_list: np.ndarray; generation of GA each latent vector is from (same order)
    """
    z_list, E_list, sr_list, lr_list, gen_list = [], [], [], [], []
    for file in files:
        with open(file, 'rb') as f:
            ld = pickle.load(f)
        if 'site_rmsd' not in ld['scores']:
            continue
        if 'lig_rmsd' not in ld['scores']:
            continue
        z_list.append(ld['z'])
        E_list.append(ld['scores']['energy'])
        sr_list.append(ld['scores']['site_rmsd'])
        lr_list.append(ld['scores']['lig_rmsd'])
        gen_list.append(ld['curr_gen'])
    z_list, E_list, sr_list, lr_list, gen_list = \
        torch.stack(z_list), np.array(E_list), np.array(sr_list), \
        np.array(lr_list), np.array(gen_list)
    return z_list, E_list, sr_list, lr_list, gen_list

print("Data loading function defined!")


Data loading function defined!


## 3. Load Data


In [24]:
# Load latent vector data
import os
data_path = '../data/pim1_4lmu_pim1_4bzo_withRL/*.pkl'

files = sorted(glob.glob(data_path))
print(f"Found {len(files)} pickle files")

# Load the data
z_list, E_list, sr_list, lr_list, gen_list = load_latents(files)

print(f"\nLoaded {len(z_list)} latent vectors")
print(f"Latent vector shape: {z_list.shape}")
print(f"Energy range: {E_list.min():.2f} - {E_list.max():.2f}")
print(f"Site RMSD range: {sr_list.min():.2f} - {sr_list.max():.2f}")
print(f"Ligand RMSD range: {lr_list.min():.2f} - {lr_list.max():.2f}")
print(f"Generation range: {gen_list.min()} - {gen_list.max()}")

# Filter to generations 0-7 as suggested by Dan (after gen 8 ignores SAE backbone)
gen_filter = gen_list <= 7
z_list_filtered = z_list[gen_filter]
E_list_filtered = E_list[gen_filter]
sr_list_filtered = sr_list[gen_filter]
lr_list_filtered = lr_list[gen_filter]
gen_list_filtered = gen_list[gen_filter]

print(f"\nAfter filtering to generations 0-7:")
print(f"Filtered samples: {len(z_list_filtered)} (from {len(z_list)})")
print(f"Generation range: {gen_list_filtered.min()} - {gen_list_filtered.max()}")

# Update variables to use filtered data
z_list, E_list, sr_list, lr_list, gen_list = z_list_filtered, E_list_filtered, sr_list_filtered, lr_list_filtered, gen_list_filtered


Found 12745 pickle files

Loaded 12745 latent vectors
Latent vector shape: torch.Size([12745, 30])
Energy range: -10821.68 - 10000.00
Site RMSD range: 0.33 - 20.90
Ligand RMSD range: 0.36 - 1217.93
Generation range: 0 - 15

After filtering to generations 0-7:
Filtered samples: 6359 (from 12745)
Generation range: 0 - 7


## 4. Analysis Strategy Implementation

Based on Dan's recommendations, plan is to:
1. **Filter to generations 0-7** (ignore 8-14 as they ignore SAE backbone)
2. **Extract key metrics** for each structure
3. **Create native-like labels** (<2Å vs >2Å ligand RMSD)
4. **Analyze Z deviations** from mean training data
5. **Correlation analysis** between Energy and ligand RMSD
6. **Identify structures** with both low energy AND low ligand RMSD


In [26]:
# Convert to numpy for easier analysis
z_np = z_list.numpy()

# 1. Create native-like labels (<2Å vs >2Å ligand RMSD)
native_like = lr_list < 2.0
not_native_like = lr_list >= 2.0

print(f"\n1. NATIVE-LIKE CLASSIFICATION:")
print(f"   Native-like (<2Å): {np.sum(native_like)} ({np.mean(native_like)*100:.1f}%)")
print(f"   Not native-like (≥2Å): {np.sum(not_native_like)} ({np.mean(not_native_like)*100:.1f}%)")

# 2. Analyze Z deviations from mean training data
z_mean = np.mean(z_np, axis=0)
z_std = np.std(z_np, axis=0)
z_deviations = z_np - z_mean  # Deviations from mean
z_normalized_deviations = z_deviations / z_std  # Normalized deviations

print(f"\n2. LATENT SPACE ANALYSIS:")
print(f"   Mean Z vector: {z_mean[:5]}... (showing first 5 dimensions)")
print(f"   Std Z vector: {z_std[:5]}... (showing first 5 dimensions)")
print(f"   Mean absolute deviation: {np.mean(np.abs(z_deviations)):.4f}")
print(f"   Mean normalized deviation: {np.mean(np.abs(z_normalized_deviations)):.4f}")

# 3. Energy vs Ligand RMSD correlation analysis
energy_rmsd_corr = np.corrcoef(E_list, lr_list)[0, 1]
print(f"\n3. ENERGY vs LIGAND RMSD CORRELATION:")
print(f"   Correlation coefficient: {energy_rmsd_corr:.3f}")
print(f"   Energy range: {E_list.min():.1f} to {E_list.max():.1f}")
print(f"   Ligand RMSD range: {lr_list.min():.2f} to {lr_list.max():.2f} Å")

# 4. Identify structures with both low energy AND low ligand RMSD
# Define thresholds: low energy (bottom 25%) and low RMSD (<2Å)
energy_threshold = np.percentile(E_list, 25)  # Bottom 25% of energies
low_energy_mask = E_list <= energy_threshold
low_rmsd_mask = lr_list < 2.0
both_good_mask = low_energy_mask & low_rmsd_mask

print(f"\n4. COMBINED SCORING (Low Energy + Low RMSD):")
print(f"   Low energy threshold (bottom 25%): {energy_threshold:.1f}")
print(f"   Low energy structures: {np.sum(low_energy_mask)} ({np.mean(low_energy_mask)*100:.1f}%)")
print(f"   Low RMSD structures: {np.sum(low_rmsd_mask)} ({np.mean(low_rmsd_mask)*100:.1f}%)")
print(f"   BOTH low energy AND low RMSD: {np.sum(both_good_mask)} ({np.mean(both_good_mask)*100:.1f}%)")

# 5. Generation-wise analysis
print(f"\n5. GENERATION-WISE ANALYSIS:")
unique_gens = sorted(np.unique(gen_list))
for gen in unique_gens:
    gen_mask = gen_list == gen
    gen_native = np.sum(native_like[gen_mask])
    gen_total = np.sum(gen_mask)
    gen_energy_mean = np.mean(E_list[gen_mask])
    gen_rmsd_mean = np.mean(lr_list[gen_mask])
    print(f"   Gen {gen}: {gen_native}/{gen_total} native-like ({gen_native/gen_total*100:.1f}%), "
          f"mean energy: {gen_energy_mean:.1f}, mean RMSD: {gen_rmsd_mean:.2f}Å")

# 6. RL Training annotation
print(f"\n6. RL TRAINING ANNOTATION:")
print(f"   This dataset: Pim1_4lmu_pim1_4bzo_withRL")
print(f"   Status: TRAINED WITH RL")
print(f"   Note: RL training may affect the distribution of structures")

# 7. Create summary DataFrame for further analysis
import pandas as pd

analysis_df = pd.DataFrame({
    'ligand_rmsd': lr_list,
    'site_rmsd': sr_list,
    'energy': E_list,
    'generation': gen_list,
    'native_like': native_like,
    'low_energy': low_energy_mask,
    'both_good': both_good_mask,
    'rl_trained': True  # This dataset was trained with RL
})

# Add Z vector dimensions
for i in range(z_np.shape[1]):
    analysis_df[f'z_{i}'] = z_np[:, i]
    analysis_df[f'z_dev_{i}'] = z_deviations[:, i]
    analysis_df[f'z_norm_dev_{i}'] = z_normalized_deviations[:, i]

print(f"\n7. ANALYSIS DATAFRAME CREATED:")
print(f"   Shape: {analysis_df.shape}")
print(f"   Columns: {list(analysis_df.columns[:10])}... (showing first 10)")


1. NATIVE-LIKE CLASSIFICATION:
   Native-like (<2Å): 595 (9.4%)
   Not native-like (≥2Å): 5764 (90.6%)

2. LATENT SPACE ANALYSIS:
   Mean Z vector: [ 0.0121443  -0.32556927 -4.6170387  -0.11062489  0.04491695]... (showing first 5 dimensions)
   Std Z vector: [1.5044806 1.0055147 2.000604  1.9965205 1.9994186]... (showing first 5 dimensions)
   Mean absolute deviation: 1.3618
   Mean normalized deviation: 0.8061

3. ENERGY vs LIGAND RMSD CORRELATION:
   Correlation coefficient: 0.133
   Energy range: -10821.7 to 10000.0
   Ligand RMSD range: 0.39 to 645.54 Å

4. COMBINED SCORING (Low Energy + Low RMSD):
   Low energy threshold (bottom 25%): -10795.6
   Low energy structures: 1590 (25.0%)
   Low RMSD structures: 595 (9.4%)
   BOTH low energy AND low RMSD: 340 (5.3%)

5. GENERATION-WISE ANALYSIS:
   Gen 0: 97/800 native-like (12.1%), mean energy: -10333.8, mean RMSD: 4.60Å
   Gen 1: 73/792 native-like (9.2%), mean energy: -6535.4, mean RMSD: 5.12Å
   Gen 2: 65/790 native-like (8.2%), mea

## 5. Key Visualizations


In [28]:
# Key visualizations for Dan's analysis strategy
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle("Dan's Analysis Strategy - Key Insights", fontsize=16, fontweight='bold')

# 1. Energy vs Ligand RMSD scatter plot
scatter = axes[0, 0].scatter(E_list, lr_list, c=gen_list, cmap='viridis', alpha=0.6)
axes[0, 0].axhline(y=2.0, color='red', linestyle='--', label='Native-like threshold (2Å)')
axes[0, 0].axvline(x=energy_threshold, color='orange', linestyle='--', label='Low energy threshold')
axes[0, 0].set_xlabel('Energy')
axes[0, 0].set_ylabel('Ligand RMSD (Å)')
axes[0, 0].set_title('Energy vs Ligand RMSD\n(Colored by Generation)')
axes[0, 0].legend()
plt.colorbar(scatter, ax=axes[0, 0], label='Generation')

# 2. Native-like vs Not native-like distribution
native_counts = [np.sum(native_like), np.sum(not_native_like)]
axes[0, 1].pie(native_counts, labels=['Native-like (<2Å)', 'Not native-like (≥2Å)'], 
               autopct='%1.1f%%', colors=['green', 'red'])
axes[0, 1].set_title('Native-like Classification')

# 3. Combined scoring: Low Energy + Low RMSD
both_good_counts = [np.sum(both_good_mask), np.sum(~both_good_mask)]
axes[0, 2].pie(both_good_counts, labels=['Both Good', 'Not Both Good'], 
               autopct='%1.1f%%', colors=['gold', 'lightgray'])
axes[0, 2].set_title('Combined Scoring\n(Low Energy + Low RMSD)')

# 4. Generation-wise native-like percentage
gen_native_pct = []
for gen in unique_gens:
    gen_mask = gen_list == gen
    gen_native_pct.append(np.mean(native_like[gen_mask]) * 100)

axes[1, 0].plot(unique_gens, gen_native_pct, 'o-', linewidth=2, markersize=8)
axes[1, 0].set_xlabel('Generation')
axes[1, 0].set_ylabel('Native-like Percentage (%)')
axes[1, 0].set_title('Native-like % by Generation')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=50, color='red', linestyle='--', alpha=0.5, label='50% line')

# 5. Z deviations from mean (first 10 dimensions)
z_dev_mean = np.mean(np.abs(z_deviations), axis=0)
axes[1, 1].bar(range(10), z_dev_mean[:10], alpha=0.7, color='purple')
axes[1, 1].set_xlabel('Latent Dimension')
axes[1, 1].set_ylabel('Mean Absolute Deviation')
axes[1, 1].set_title('Z Deviations from Mean\n(First 10 Dimensions)')
axes[1, 1].grid(True, alpha=0.3)

# 6. Energy distribution by native-like status
native_energies = E_list[native_like]
not_native_energies = E_list[not_native_like]

axes[1, 2].hist(native_energies, bins=50, alpha=0.7, label='Native-like', color='green', density=True)
axes[1, 2].hist(not_native_energies, bins=50, alpha=0.7, label='Not native-like', color='red', density=True)
axes[1, 2].set_xlabel('Energy')
axes[1, 2].set_ylabel('Density')
axes[1, 2].set_title('Energy Distribution by\nNative-like Status')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print key insights
print("\n" + "="*60)
print("KEY INSIGHTS FROM DAN'S ANALYSIS")
print("="*60)

print(f"\n🎯 NATIVE-LIKE STRUCTURES:")
print(f"   • {np.sum(native_like)} structures are native-like (<2Å)")
print(f"   • {np.mean(native_like)*100:.1f}% of all structures are native-like")

print(f"\n⚡ ENERGY vs RMSD RELATIONSHIP:")
print(f"   • Correlation: {energy_rmsd_corr:.3f}")
if abs(energy_rmsd_corr) < 0.3:
    print(f"   • WEAK correlation - Energy is imperfect predictor of RMSD")
else:
    print(f"   • MODERATE correlation - Energy has some predictive power")

print(f"\n🏆 COMBINED SCORING:")
print(f"   • {np.sum(both_good_mask)} structures have BOTH low energy AND low RMSD")
print(f"   • {np.mean(both_good_mask)*100:.1f}% meet both criteria")
print(f"   • These are the PRIORITY structures for analysis")

print(f"\n📈 GENERATION PROGRESS:")
best_gen = unique_gens[np.argmax(gen_native_pct)]
worst_gen = unique_gens[np.argmin(gen_native_pct)]
print(f"   • Best generation: {best_gen} ({gen_native_pct[np.argmax(gen_native_pct)]:.1f}% native-like)")
print(f"   • Worst generation: {worst_gen} ({gen_native_pct[np.argmin(gen_native_pct)]:.1f}% native-like)")

print(f"\n🧠 LATENT SPACE INSIGHTS:")
print(f"   • Mean absolute deviation: {np.mean(np.abs(z_deviations)):.4f}")
print(f"   • Most variable dimension: {np.argmax(z_dev_mean)} (deviation: {np.max(z_dev_mean):.4f})")
print(f"   • Least variable dimension: {np.argmin(z_dev_mean)} (deviation: {np.min(z_dev_mean):.4f})")

print("\n" + "="*60)



KEY INSIGHTS FROM DAN'S ANALYSIS

🎯 NATIVE-LIKE STRUCTURES:
   • 595 structures are native-like (<2Å)
   • 9.4% of all structures are native-like

⚡ ENERGY vs RMSD RELATIONSHIP:
   • Correlation: 0.133
   • WEAK correlation - Energy is imperfect predictor of RMSD

🏆 COMBINED SCORING:
   • 340 structures have BOTH low energy AND low RMSD
   • 5.3% meet both criteria
   • These are the PRIORITY structures for analysis

📈 GENERATION PROGRESS:
   • Best generation: 0 (12.1% native-like)
   • Worst generation: 3 (8.1% native-like)

🧠 LATENT SPACE INSIGHTS:
   • Mean absolute deviation: 1.3618
   • Most variable dimension: 26 (deviation: 1.8788)
   • Least variable dimension: 1 (deviation: 0.8036)



## 4. Basic Data Statistics


In [15]:
# Convert to numpy for easier analysis
z_np = z_list.numpy()

# Basic statistics
print("="*50)
print("BASIC DATA STATISTICS")
print("="*50)

print(f"Dataset shape: {z_np.shape}")
print(f"Number of generations: {len(np.unique(gen_list))}")
print(f"Unique generations: {sorted(np.unique(gen_list))}")

# Quality classification
good_poses = lr_list < 2.0
poor_poses = lr_list >= 3.0
intermediate_poses = (lr_list >= 2.0) & (lr_list < 3.0)

print(f"\nQuality Distribution:")
print(f"  Good poses (<2Å): {np.sum(good_poses)} ({np.mean(good_poses)*100:.1f}%)")
print(f"  Poor poses (≥3Å): {np.sum(poor_poses)} ({np.mean(poor_poses)*100:.1f}%)")
print(f"  Intermediate poses (2-3Å): {np.sum(intermediate_poses)} ({np.mean(intermediate_poses)*100:.1f}%)")

# Latent space statistics
print(f"\nLatent Space Statistics:")
print(f"  Mean: {np.mean(z_np):.4f}")
print(f"  Std: {np.std(z_np):.4f}")
print(f"  Min: {np.min(z_np):.4f}")
print(f"  Max: {np.max(z_np):.4f}")
print(f"  Sparsity (|z| < 0.1): {np.mean(np.abs(z_np) < 0.1)*100:.1f}%")


BASIC DATA STATISTICS
Dataset shape: (12745, 30)
Number of generations: 16
Unique generations: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Quality Distribution:
  Good poses (<2Å): 1363 (10.7%)
  Poor poses (≥3Å): 11167 (87.6%)
  Intermediate poses (2-3Å): 215 (1.7%)

Latent Space Statistics:
  Mean: -0.8827
  Std: 2.3633
  Min: -10.6605
  Max: 10.1885
  Sparsity (|z| < 0.1): 3.3%


## 7. Individual Visualizations for Slides


In [33]:
# 1. Energy vs Ligand RMSD Scatter Plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(E_list, lr_list, c=gen_list, cmap='viridis', alpha=0.6, s=20)
plt.axhline(y=2.0, color='red', linestyle='--', linewidth=2, label='Native-like threshold (2Å)')
plt.axvline(x=energy_threshold, color='orange', linestyle='--', linewidth=2, label='Low energy threshold')
plt.xlabel('Energy', fontsize=14)
plt.ylabel('Ligand RMSD (Å)', fontsize=14)
plt.title('Energy vs Ligand RMSD\n(Colored by Generation)', fontsize=16, fontweight='bold')
plt.legend(fontsize=12)
plt.colorbar(scatter, label='Generation')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {energy_rmsd_corr:.3f}")
print(f"Total structures: {len(E_list):,}")
print(f"Energy range: {E_list.min():.1f} to {E_list.max():.1f}")
print(f"Ligand RMSD range: {lr_list.min():.2f} to {lr_list.max():.2f} Å")


Correlation coefficient: 0.133
Total structures: 6,359
Energy range: -10821.7 to 10000.0
Ligand RMSD range: 0.39 to 645.54 Å


In [35]:
# 2. Native-like Classification Pie Chart
plt.figure(figsize=(8, 8))
native_counts = [np.sum(native_like), np.sum(not_native_like)]
labels = ['Native-like (<2Å)', 'Not native-like (≥2Å)']
colors = ['#2E8B57', '#DC143C']  # Sea green and crimson
explode = (0.05, 0)  # Slightly separate the native-like slice

wedges, texts, autotexts = plt.pie(native_counts, labels=labels, autopct='%1.1f%%', 
                                   colors=colors, explode=explode, startangle=90,
                                   textprops={'fontsize': 14})

# Make percentage text bold
for autotext in autotexts:
    autotext.set_fontweight('bold')
    autotext.set_fontsize(16)

plt.title('Native-like Classification\n(6,359 structures)', fontsize=18, fontweight='bold', pad=20)
plt.axis('equal')
plt.tight_layout()
plt.show()

print(f"Native-like structures: {np.sum(native_like):,} ({np.mean(native_like)*100:.1f}%)")
print(f"Not native-like structures: {np.sum(not_native_like):,} ({np.mean(not_native_like)*100:.1f}%)")


Native-like structures: 595 (9.4%)
Not native-like structures: 5,764 (90.6%)


In [36]:
# 3. Generation-wise Native-like Percentage
plt.figure(figsize=(10, 6))
gen_native_pct = []
for gen in unique_gens:
    gen_mask = gen_list == gen
    gen_native_pct.append(np.mean(native_like[gen_mask]) * 100)

plt.plot(unique_gens, gen_native_pct, 'o-', linewidth=3, markersize=10, 
         color='#4169E1', markerfacecolor='white', markeredgewidth=2)
plt.axhline(y=50, color='red', linestyle='--', alpha=0.7, linewidth=2, label='50% benchmark')
plt.xlabel('Generation', fontsize=14)
plt.ylabel('Native-like Percentage (%)', fontsize=14)
plt.title('Native-like Success Rate by Generation\n(RL-Trained Optimization)', fontsize=16, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.ylim(0, max(gen_native_pct) * 1.1)

# Add value labels on points
for i, (gen, pct) in enumerate(zip(unique_gens, gen_native_pct)):
    plt.annotate(f'{pct:.1f}%', (gen, pct), textcoords="offset points", 
                xytext=(0,10), ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("Generation-wise Native-like Performance:")
for gen, pct in zip(unique_gens, gen_native_pct):
    gen_mask = gen_list == gen
    gen_count = np.sum(gen_mask)
    native_count = np.sum(native_like[gen_mask])
    print(f"  Gen {gen}: {native_count}/{gen_count} ({pct:.1f}%)")

best_gen = unique_gens[np.argmax(gen_native_pct)]
worst_gen = unique_gens[np.argmin(gen_native_pct)]
print(f"\nBest generation: {best_gen} ({gen_native_pct[np.argmax(gen_native_pct)]:.1f}%)")
print(f"Worst generation: {worst_gen} ({gen_native_pct[np.argmin(gen_native_pct)]:.1f}%)")


Generation-wise Native-like Performance:
  Gen 0: 97/800 (12.1%)
  Gen 1: 73/792 (9.2%)
  Gen 2: 65/790 (8.2%)
  Gen 3: 64/787 (8.1%)
  Gen 4: 70/797 (8.8%)
  Gen 5: 71/796 (8.9%)
  Gen 6: 75/797 (9.4%)
  Gen 7: 80/800 (10.0%)

Best generation: 0 (12.1%)
Worst generation: 3 (8.1%)


## 5. Data Distribution Visualizations


In [ ]:
# Create comprehensive distribution plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Data Distribution Analysis', fontsize=16, fontweight='bold')

# Energy distribution
axes[0, 0].hist(E_list, bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Energy Distribution')
axes[0, 0].set_xlabel('Energy')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(np.mean(E_list), color='red', linestyle='--', label=f'Mean: {np.mean(E_list):.2f}')
axes[0, 0].legend()

# Site RMSD distribution
axes[0, 1].hist(sr_list, bins=50, alpha=0.7, edgecolor='black', color='orange')
axes[0, 1].set_title('Site RMSD Distribution')
axes[0, 1].set_xlabel('Site RMSD (Å)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(np.mean(sr_list), color='red', linestyle='--', label=f'Mean: {np.mean(sr_list):.2f}')
axes[0, 1].legend()

# Ligand RMSD distribution
axes[0, 2].hist(lr_list, bins=50, alpha=0.7, edgecolor='black', color='green')
axes[0, 2].set_title('Ligand RMSD Distribution')
axes[0, 2].set_xlabel('Ligand RMSD (Å)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].axvline(2.0, color='red', linestyle='--', label='Good/Bad threshold')
axes[0, 2].axvline(3.0, color='red', linestyle='--', label='Bad threshold')
axes[0, 2].legend()

# Generation distribution
axes[1, 0].hist(gen_list, bins=len(np.unique(gen_list)), alpha=0.7, edgecolor='black', color='purple')
axes[1, 0].set_title('Generation Distribution')
axes[1, 0].set_xlabel('Generation')
axes[1, 0].set_ylabel('Frequency')

# Energy vs Ligand RMSD scatter
scatter = axes[1, 1].scatter(E_list, lr_list, alpha=0.6, c=gen_list, cmap='viridis')
axes[1, 1].set_title('Energy vs Ligand RMSD')
axes[1, 1].set_xlabel('Energy')
axes[1, 1].set_ylabel('Ligand RMSD (Å)')
plt.colorbar(scatter, ax=axes[1, 1], label='Generation')

# Site RMSD vs Ligand RMSD scatter
scatter = axes[1, 2].scatter(sr_list, lr_list, alpha=0.6, c=gen_list, cmap='viridis')
axes[1, 2].set_title('Site RMSD vs Ligand RMSD')
axes[1, 2].set_xlabel('Site RMSD (Å)')
axes[1, 2].set_ylabel('Ligand RMSD (Å)')
plt.colorbar(scatter, ax=axes[1, 2], label='Generation')

plt.tight_layout()
plt.show()


KeyboardInterrupt: 

: 

## 6. Latent Space Analysis


In [30]:
# Analyze latent dimensions
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Latent Space Analysis', fontsize=16, fontweight='bold')

# PCA visualization
pca = PCA(n_components=2)
z_pca = pca.fit_transform(z_np)

scatter = axes[0, 0].scatter(z_pca[:, 0], z_pca[:, 1], c=lr_list, cmap='viridis', alpha=0.6)
axes[0, 0].set_title('Latent Space (PCA) - Colored by Ligand RMSD')
axes[0, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[0, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.colorbar(scatter, ax=axes[0, 0], label='Ligand RMSD (Å)')

# PCA colored by generation
scatter = axes[0, 1].scatter(z_pca[:, 0], z_pca[:, 1], c=gen_list, cmap='plasma', alpha=0.6)
axes[0, 1].set_title('Latent Space (PCA) - Colored by Generation')
axes[0, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[0, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.colorbar(scatter, ax=axes[0, 1], label='Generation')

# Latent dimension distributions (first 4 dimensions)
for i in range(min(4, z_np.shape[1])):
    row = 1
    col = i
    axes[row, col].hist(z_np[:, i], bins=50, alpha=0.7, edgecolor='black')
    axes[row, col].set_title(f'Latent Dimension {i}')
    axes[row, col].set_xlabel(f'Value')
    axes[row, col].set_ylabel('Frequency')
    axes[row, col].axvline(np.mean(z_np[:, i]), color='red', linestyle='--', 
                          label=f'Mean: {np.mean(z_np[:, i]):.3f}')
    axes[row, col].legend()

plt.tight_layout()
plt.show()

print(f"PCA explained variance: PC1={pca.explained_variance_ratio_[0]:.3f}, PC2={pca.explained_variance_ratio_[1]:.3f}")
print(f"Total explained variance (first 2 components): {np.sum(pca.explained_variance_ratio_[:2]):.3f}")


IndexError: index 2 is out of bounds for axis 1 with size 2

## 7. Correlation Analysis


In [31]:
# Create correlation analysis
data_dict = {
    'energy': E_list,
    'site_rmsd': sr_list,
    'ligand_rmsd': lr_list,
    'generation': gen_list
}

# Add first 10 latent dimensions for correlation
for i in range(min(10, z_np.shape[1])):
    data_dict[f'latent_{i}'] = z_np[:, i]

df = pd.DataFrame(data_dict)
correlation_matrix = df.corr()

# Find strongest correlations with ligand RMSD
ligand_rmsd_corr = correlation_matrix['ligand_rmsd'].abs().sort_values(ascending=False)

print("Top correlations with Ligand RMSD:")
for i, (feature, corr) in enumerate(ligand_rmsd_corr.head(10).items()):
    print(f"  {i+1:2d}. {feature:15s}: {corr:.3f}")

# Create correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'label': 'Correlation'})
plt.title('Correlation Matrix: Metrics and Top Latent Dimensions')
plt.tight_layout()
plt.show()


Top correlations with Ligand RMSD:
   1. ligand_rmsd    : 1.000
   2. site_rmsd      : 0.257
   3. energy         : 0.133
   4. latent_2       : 0.085
   5. latent_5       : 0.081
   6. latent_9       : 0.067
   7. latent_3       : 0.056
   8. latent_1       : 0.032
   9. latent_6       : 0.027
  10. latent_7       : 0.018


## 8. Summary and Conclusions


In [32]:
# Generate summary report
print("="*60)
print("LATENT ANALYSIS SUMMARY")
print("="*60)

print(f"\nDataset Overview:")
print(f"  • Total samples: {len(z_list):,}")
print(f"  • Latent dimensions: {z_np.shape[1]}")
print(f"  • Generations: {len(np.unique(gen_list))} ({np.min(gen_list)}-{np.max(gen_list)})")

print(f"\nQuality Distribution:")
print(f"  • Good poses (<2Å): {np.sum(good_poses):,} ({np.mean(good_poses)*100:.1f}%)")
print(f"  • Poor poses (≥3Å): {np.sum(poor_poses):,} ({np.mean(poor_poses)*100:.1f}%)")
print(f"  • Intermediate poses (2-3Å): {np.sum(intermediate_poses):,} ({np.mean(intermediate_poses)*100:.1f}%)")

print(f"\nKey Correlations with Ligand RMSD:")
print(f"  • Energy: {correlation_matrix.loc['energy', 'ligand_rmsd']:.3f}")
print(f"  • Site RMSD: {correlation_matrix.loc['site_rmsd', 'ligand_rmsd']:.3f}")
print(f"  • Generation: {correlation_matrix.loc['generation', 'ligand_rmsd']:.3f}")

print(f"\nLatent Space Properties:")
print(f"  • Mean: {np.mean(z_np):.4f}")
print(f"  • Standard deviation: {np.std(z_np):.4f}")
print(f"  • Sparsity (|z| < 0.1): {np.mean(np.abs(z_np) < 0.1)*100:.1f}%")
print(f"  • PCA explained variance (PC1+PC2): {np.sum(pca.explained_variance_ratio_[:2]):.3f}")

print("\n" + "="*60)
print("Analysis completed successfully!")
print("="*60)


LATENT ANALYSIS SUMMARY

Dataset Overview:
  • Total samples: 6,359
  • Latent dimensions: 30
  • Generations: 8 (0-7)

Quality Distribution:
  • Good poses (<2Å): 1,363 (10.7%)
  • Poor poses (≥3Å): 11,167 (87.6%)
  • Intermediate poses (2-3Å): 215 (1.7%)

Key Correlations with Ligand RMSD:
  • Energy: 0.133
  • Site RMSD: 0.257
  • Generation: 0.017

Latent Space Properties:
  • Mean: -0.8283
  • Standard deviation: 2.3723
  • Sparsity (|z| < 0.1): 3.4%
  • PCA explained variance (PC1+PC2): 0.451

Analysis completed successfully!
